In [6]:
import sys
import struct
import math
import time
import datetime
import argparse
import pandas as pd
import uuid
import requests #ICAO Airport Code API

# Constants and Definitions
HEADER_LEN = 8
SIGNATURE_LEN = 2
PRIMARY_FLIGHT_DATA_LEN = 24
RDAC1_DATA_LEN = 56
RDAC1_DATA_LEN_REV_2 = 63
RDAC2_DATA_LEN = RDAC1_DATA_LEN
RDAC2_DATA_LEN_REV_2 = RDAC1_DATA_LEN_REV_2
ATTITUDE_DATA_LEN = 14
GPS_DATA_LEN_REV_0 = 20
GPS_DATA_LEN_REV_1 = 24

MAX_PACKET = 1000  # larger than any legal packet
MAXIMUM_LOG_INTERVAL = 30  #Time between MGL log records.

PACKET_SIGNATURE_1 = 0xaa
PACKET_SIGNATURE_2 = 0x55
SIGNATURE_LEN = 2

TIMESTAMP_LEN = 4

MAX_REVISION = 3  # counting from zero

delimiters = {
    "tab": "\t",
    "comma": ",",
    "space": " ",
    "no_delim": "",
}

fudge_factors = [6, 2, 2, 6]  # length is MAX_REVISION+1 = 4

primary_section = 0
rdac1_section = 1
rdac2_section = 2
attitude_section = 3
gps_section = 4

section_sizes = [
    [24, 24, 24, 24],  # primary
    [56, 56, 63, 63],  # rdac1
    [56, 56, 63, 63],  # rdac2
    [14, 14, 14, 14],  # attitude
    [20, 24, 24, 24],  # GPS
]

rdac1_data_tag = 1
rdac2_data_tag = 2
attitude_data_tag = 3
gps_data_tag = 4

RDAC1_PRESENT = 1 << rdac1_data_tag  # (1 << 1) == 2
RDAC2_PRESENT = 1 << rdac2_data_tag  # (1 << 2) == 4
ATTITUDE_PRESENT = 1 << attitude_data_tag  # (1 << 3) == 8
GPS_PRESENT = 1 << gps_data_tag  # (1 << 4) == 16


class EnigmaLogParser:
    def __init__(self, filename):
        self.logfile = open(filename, 'rb')
        self.bad_packets = 0
        self.reversals = 0

    def read_packet(self):
        packet = bytearray()
        processed = 0
        while True:
            i1 = self.logfile.read(1)
            if not i1:
                print("rp_EOF_i1", end='')    
                return None  # EOF
            if i1[0] == PACKET_SIGNATURE_1:
                i2 = self.logfile.read(1)
                if not i2:
                    print("rp_EOF_i2", end='')      
                    return None  # EOF
                if i2[0] == PACKET_SIGNATURE_2:
                    # Valid starting signature, read rest of packet
                    packet_body = self.read_body()
                    if packet_body is None:
                        return None
                    packet.extend(i1)
                    packet.extend(i2)
                    packet.extend(packet_body)
                    return packet
                else:
                    # Move back one byte
                    self.logfile.seek(-1, 1)
            processed += 1
            
        return None  # Should not reach here

    def read_body(self):
        packet_body = bytearray()
        processed = 0
        while True:
            i1 = self.logfile.read(1)
            if not i1:
                break  # EOF
            if i1[0] == PACKET_SIGNATURE_1:
                i2 = self.logfile.read(1)
                if not i2:
                    break  # EOF
                if i2[0] == PACKET_SIGNATURE_2:
                    # Signature of next packet, back up and exit
                    self.logfile.seek(-2, 1)
                    break
                else:
                    # False trigger, save byte and continue
                    packet_body.append(i1[0])
                    self.logfile.seek(-1, 1)
            else:
                packet_body.append(i1[0])
            processed += 1
            if processed % MAX_PACKET == 0 and i1[0] == 0: 
                print(f'b-{processed}-{i1[0]:02X} End of data at:{self.logfile.tell()}')
                i1 = None
                break  # End of data
  
        return packet_body

    def read_and_id_packet(self):
        while True:
            packet = self.read_packet()
            if packet is None:
                print("rp_none", end='')
                return None
            packet_len = len(packet)
            if packet_len < HEADER_LEN:
                continue  # Try next packet

            timestamp = (
                packet[7] << 24 |
                packet[6] << 16 |
                packet[5] << 8 |
                packet[4]
            )
            # Extract bytes from index 4 to 7 (inclusive)
            bytes_4 = packet[3:7]  # Slices from index 3 to 6
            # Convert the extracted 4 bytes to an integer (big-endian or little-endian)
            #timestamp = int.from_bytes(bytes_4, byteorder='little') # 'big' or 'little'
            
            payload = packet
            # packet[0] is the total payload length + fudge factor
            # packet[1] is an offset
            for revision in range(MAX_REVISION + 1):
                sections_seen = 0
                rdac1_len = 0
                rdac2_len = 0
                attitude_len = 0
                gps_len = 0

                if revision >= len(fudge_factors):
                    print("Revisions greater than possible")
                    continue

                if packet[0] < fudge_factors[revision]:
                    print("Packet less than fudge factor")
                    continue

                payload_len = packet[2] + SIGNATURE_LEN

                if (payload_len < PRIMARY_FLIGHT_DATA_LEN or
                        payload_len != packet_len):
                    print(f"Packet length error. Payload:{payload_len}, Packet:{packet_len}")
                    continue

                p_offset = section_sizes[0][revision]+ HEADER_LEN + fudge_factors[revision]
                if p_offset < 0 or p_offset > len(payload):
                    print("Invalid offset")
                    continue  # Invalid offset

                p_idx = p_offset

                # Parse sections
                if p_idx < payload_len and payload[p_idx] == rdac1_data_tag:
                    sections_seen |= RDAC1_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    rdac1_len = payload[p_idx]
                    p_idx += 1
                    p_idx += rdac1_len

                if p_idx < payload_len and payload[p_idx] == rdac2_data_tag:
                    sections_seen |= RDAC2_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    rdac2_len = payload[p_idx]
                    p_idx += 1
                    p_idx += rdac2_len

                if p_idx < payload_len and payload[p_idx] == attitude_data_tag:
                    sections_seen |= ATTITUDE_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    attitude_len = payload[p_idx]
                    p_idx += 1
                    p_idx += attitude_len

                if p_idx < payload_len and payload[p_idx] == gps_data_tag:
                    sections_seen |= GPS_PRESENT
                    tag = payload[p_idx]
                    p_idx += 1
                    gps_len = payload[p_idx]
                    p_idx += 1
                    p_idx += gps_len

                if (p_idx == payload_len and
                    ((sections_seen & RDAC1_PRESENT == 0) or rdac1_len == section_sizes[rdac1_section][revision]) and
                    ((sections_seen & RDAC2_PRESENT == 0) or rdac2_len == section_sizes[rdac2_section][revision]) and
                    ((sections_seen & ATTITUDE_PRESENT == 0) or attitude_len == section_sizes[attitude_section][revision]) and
                    ((sections_seen & GPS_PRESENT == 0) or gps_len == section_sizes[gps_section][revision])
                   ):
                    #print(payload, payload_len, revision, sections_seen)
                    return timestamp, payload[HEADER_LEN:], payload_len, revision, sections_seen

            self.bad_packets += 1
            print("Pcket error try next packet")
            # Try next packet

   
    #Determine sections in a record based on first section only
    
    def determine_sections(self):
        self.logfile.seek(0, 0)
        cur_pos = self.logfile.tell()
        result = self.read_and_id_packet()
        if result is None:
            print("sec_break", end='')
            sections_seen = 0
        else:
            timestamp, payload, payload_len, revision, sections_seen = result
        return sections_seen, timestamp

    def rdac_headers(self):
        headers = ["RPM", "rfl1", "rfl2", "ch1", "ch2", "flow", "MAP", "fl1", "fl2", "flc",
                   "o_t", "oil_p", "carb", "f_p", "h2o"]
        headers.extend([f"tc{i}" for i in range(1, 13)])
        headers.extend(["rch1", "rch2", "rot", "rop", "ref", "fail"])
        return headers

    def print_header(self, delimiter_type, sections_seen):
        delimiter = delimiters[delimiter_type]
        headers = []
        headers.append("Time")

        headers.extend(["GUID","alt", "baro", "ASI", "TAS", "VSI", "g_s", "rotor", "m_v", "b_v", "amps", "AOA", "OAT"])

        if sections_seen & RDAC1_PRESENT:
            headers.extend(self.rdac_headers())
        if sections_seen & RDAC2_PRESENT:
            headers.extend(self.rdac_headers())
        if sections_seen & ATTITUDE_PRESENT:
            headers.extend(["bank", "ptch", "slip", "mgl_hdg", "yaw", "G", "turn"])
        if sections_seen & GPS_PRESENT:
            headers.extend(["lat", "long", "gps_hdg", "gs", "g_alt", "status", "sats", "hac", "vac"])

        print(delimiter.join(headers))
        file.write(delimiter.join(headers) + "\n")  # Write headers to the file
        return headers


    def process_packets(self, delimiter_type, stop_pos, sections_seen, start_time):
        delimiter = delimiters[delimiter_type]
        packets = 0
        mgl_records = []
        mgl_flight = []
        current_time = start_time
        while self.logfile.tell() < stop_pos:
            result = self.read_and_id_packet()
            if result is None:
                break
            timestamp, payload, payload_len, revision, packet_sections_seen = result
            p = payload
            idx = 0
            valid = True
            #print(f'T:{timestamp} - Len:{payload_len} - Rev:{revision}')
            # Time processing
            if revision >= 2:
                base_time = datetime.datetime(2000, 1, 1)
                adj_time = base_time + datetime.timedelta(seconds=timestamp)
                time_str = adj_time.strftime("%Y/%m/%d %H:%M:%S")
                output = [f'"{time_str}"']
            else:
                output = [str(timestamp)]
            # Sort packets into flights
            if timestamp == current_time or timestamp > current_time + MAXIMUM_LOG_INTERVAL:
                flight = []
                # Generate a GUID for each flight
                flight_guid = str(uuid.uuid4())
                mgl_flight.append([flight_guid, packets])
                print(f"Flight: {flight_guid} @ {time_str}")
            current_time = timestamp
            # Primary Flight Data
            # Using struct.unpack for binary data parsing
            try:
                #Add Flight Guide to Output
                output.append(str(flight_guid))
                # Altitude (S32)
                alt, = struct.unpack_from('<i', p, idx)
                idx += 4
                output.append(str(alt))
                # Barometer (S16)
                baro, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(baro))
                # Airspeed (S16)
                asi, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(asi))
                # TAS (S16)
                tas, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(tas))
                # VSI (S16)
                vsi, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(vsi))
                # Glide Slope (S16 tenths)
                gs, = struct.unpack_from('<h', p, idx)
                idx += 2
                gs_tenths = gs / 10.0
                output.append(f"{gs_tenths:.1f}")
                # Rotor RPM (U16)
                rotor_raw, = struct.unpack_from('<H', p, idx)
                idx += 2
                rotor = rotor_raw & 0x7fff
                output.append(str(rotor))
                # Main Voltage (U8 tenths)
                mv_raw, = struct.unpack_from('<B', p, idx)
                idx += 1
                mv = mv_raw / 10.0
                output.append(f"{mv:.1f}")
                # Backup Voltage (U8 tenths)
                bv_raw, = struct.unpack_from('<B', p, idx)
                idx += 1
                bv = bv_raw / 10.0
                output.append(f"{bv:.1f}")
                # Current (S16 tenths)
                amps_raw, = struct.unpack_from('<h', p, idx)
                idx += 2
                amps = amps_raw / 10.0
                output.append(f"{amps:.1f}")
                # AOA (S16)
                aoa, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(aoa))
                # OAT (S16)
                oat, = struct.unpack_from('<h', p, idx)
                idx += 2
                output.append(str(oat))
            except struct.error:
                self.bad_packets += 1
                continue  # Skip to next packet
            idx += fudge_factors[revision] #Seem to need to skip some extra bytes here
            # RDAC Sections
            for rdac_tag in [rdac1_data_tag, rdac2_data_tag]:
                if sections_seen & (1 << rdac_tag):
                    if idx < len(p) and p[idx] == rdac_tag:
                        # Unpack tag and length
                        tag, length = struct.unpack_from('<BB', p, idx)
                        idx += 2  # Move index past tag and length

                        # Direct use of struct.unpack_from for each data field
                        # RPM (16-bit unsigned)
                        rpm = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(rpm))

                        # Tank 1 raw (16-bit unsigned)
                        tank1_raw = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(tank1_raw))

                        # Tank 2 raw (16-bit unsigned)
                        tank2_raw = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(tank2_raw))

                        # Rotax CHT 1 (16-bit unsigned)
                        rotax_cht1 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(rotax_cht1))

                        # Rotax CHT 2 (16-bit unsigned)
                        rotax_cht2 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(rotax_cht2))

                        # Fuel flow (16-bit unsigned, divided by 100)
                        fuel_flow = struct.unpack_from('<H', p, idx)[0] / 100.0 if valid else 0.0
                        idx += 2
                        output.append(f"{fuel_flow:.2f}")

                        # MAP (Manifold Absolute Pressure, 16-bit unsigned)
                        map_val = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(map_val))

                        # Fuel level 1 (16-bit unsigned)
                        fuel_level1 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(fuel_level1))

                        # Fuel level 2 (16-bit unsigned)
                        fuel_level2 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(fuel_level2))

                        # Calculated fuel level (16-bit unsigned, divided by 10)
                        calc_fuel_level = struct.unpack_from('<H', p, idx)[0] / 10.0 if valid else 0.0
                        idx += 2
                        output.append(f"{calc_fuel_level:.1f}")

                        # Oil temperature (16-bit unsigned)
                        oil_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(oil_temp))

                        # Oil pressure (16-bit unsigned, divided by 10)
                        oil_pressure = struct.unpack_from('<H', p, idx)[0] / 10.0 if valid else 0.0
                        idx += 2
                        output.append(f"{oil_pressure:.1f}")

                        # Carb temperature (16-bit signed)
                        carb_temp = struct.unpack_from('<h', p, idx)[0] if valid else 0
                        idx += 2
                        output.append(str(carb_temp))

                        # Fuel pressure (8-bit unsigned, divided by 10)
                        fuel_pressure = struct.unpack_from('<B', p, idx)[0] / 10.0 if valid else 0.0
                        idx += 1
                        output.append(f"{fuel_pressure:.1f}")

                        # Water temperature (8-bit unsigned)
                        water_temp = struct.unpack_from('<B', p, idx)[0] if valid else 0
                        idx += 1
                        output.append(str(water_temp))

                        # Process 12 thermocouples (each 16-bit unsigned)
                        for i in range(12):
                            thermocouple = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(thermocouple))

                        # Additional fields if revision == 2
                        if revision == 2:
                            raw_cht1 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_cht1))

                            raw_cht2 = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_cht2))

                            raw_oil_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_oil_temp))

                            raw_oil_pressure = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(raw_oil_pressure))

                            rdac_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            #Result Doesn't compare to the MGL program e.g 2403 vs 21. Reads the bits correctly.
                            idx += 2
                            output.append(str(rdac_temp))

                            rdac_fail = struct.unpack_from('<B', p, idx)[0] if valid else 0
                            idx += 1
                            output.append(str(rdac_fail))

                        else:
                            # If revision is not 2, treat raw CHT and oil fields as invalid (set valid=False)
                            raw_cht1 = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_cht1))

                            raw_cht2 = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_cht2))

                            raw_oil_temp = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_oil_temp))

                            raw_oil_pressure = struct.unpack_from('<H', p, idx)[0] if False else 0
                            idx += 2
                            output.append(str(raw_oil_pressure))

                            rdac_temp = struct.unpack_from('<H', p, idx)[0] if valid else 0
                            idx += 2
                            output.append(str(rdac_temp))

                            rdac_fail = struct.unpack_from('<B', p, idx)[0] if valid else 0
                            idx += 1
                            output.append(str(rdac_fail))

                            # Move past padding byte if present
                            idx += 1
                    else:
                        output.extend(['3'] * 39)

            # Attitude Section
            if sections_seen & ATTITUDE_PRESENT:
                if idx < len(p) and p[idx] == attitude_data_tag:
                    # Unpack tag and length
                    tag, length = struct.unpack_from('<BB', p, idx)
                    idx += 2  # Move past tag and length

                    # Bank angle (signed 16-bit integer)
                    bank = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(bank))

                    # Pitch angle (signed 16-bit integer)
                    pitch = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(pitch))

                    # Slip (signed 16-bit integer)
                    slip = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(slip))

                    # Magnetic heading (signed 16-bit integer)
                    magnetic_heading = struct.unpack_from('<h', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(magnetic_heading))

                    # Yaw/gyro heading (unsigned 16-bit integer)
                    yaw_gyro_heading = struct.unpack_from('<H', p, idx)[0] if valid else 0
                    idx += 2
                    output.append(str(yaw_gyro_heading))

                    # G-force (signed 8-bit integer, interpreted in tenths)
                    g_force = struct.unpack_from('<b', p, idx)[0] / 10.0 if valid else 0.0
                    idx += 1
                    output.append(f"{g_force:.1f}")

                    # Turn rate (Z-format MMSS: unsigned byte of seconds followed by signed word of minutes)
                    turn_rate_seconds = struct.unpack_from('<B', p, idx)[0] if valid else 0  # unsigned 8-bit integer for seconds
                    idx += 1
                    turn_rate_minutes = struct.unpack_from('<h', p, idx)[0] if valid else 0  # signed 16-bit integer for minutes
                    idx += 2

                    # Format and append turn rate in MM:SS format
                    turn_rate = f"{turn_rate_minutes}:{turn_rate_seconds:02d}"
                    output.append(turn_rate)
                    
                else:
                    output.extend(['7'] * 7)

            # GPS Section
            required_size = idx + 2  # Current offset + size of 4 bytes for '<f'
            if len(p) > required_size & GPS_PRESENT:
                if idx < len(p) and p[idx] == gps_data_tag:          
                    # Unpack GPS tag and length (both are unsigned 8-bit integers)
                    tag, length = struct.unpack_from('<BB', p, idx)
                    idx += 2  # Move index past tag and length

                    # Interpret latitude as a 32-bit float (4 bytes)
                    latitude, = struct.unpack_from('<f', p, idx)
                    idx += 4
                    output.append(f"{latitude:.5f}")  # Append latitude to output

                    # Interpret longitude as a 32-bit float (4 bytes), negated (multiply by -1)
                    longitude, = struct.unpack_from('<f', p, idx)
                    idx += 4
                    output.append(f"{longitude:.5f}")  # Append negated longitude

                    # Interpret GPS heading as signed 32-bit integer
                    heading, = struct.unpack_from('<i', p, idx)
                    idx += 4
                    output.append(str(heading))  # Append GPS heading to output

                    # Interpret ground speed as signed 32-bit integer
                    ground_speed, = struct.unpack_from('<i', p, idx)
                    idx += 4
                    output.append(str(ground_speed))  # Append ground speed to output

                    # Interpret altitude as signed 32-bit integer
                    altitude, = struct.unpack_from('<i', p, idx)
                    idx += 4
                    output.append(str(altitude))  # Append altitude to output

                    # GPS status, satellites, horizontal accuracy, vertical accuracy are 8-bit integers
                    if revision > 0:  # These fields are valid only for certain revisions
                        gps_status, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(gps_status))  # Append GPS status to output

                        satellites, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(satellites))  # Append satellites count to output

                        horz_accuracy, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(horz_accuracy))  # Append horizontal accuracy to output

                        vert_accuracy, = struct.unpack_from('<b', p, idx)
                        idx += 1
                        output.append(str(vert_accuracy))  # Append vertical accuracy to output

                else:
                    output.extend(['9'] * 9)       
            #print(delimiter.join(output))
            #print(output)
            file.write(delimiter.join(output) + "\n")  # Write packet to the file
            mgl_records.append(output)
            packets += 1
            if packets % 1000 == 0:
                print("p", end='')
            
        return mgl_records, mgl_flight
    
#extract the flight detals from the flight records

from math import radians, sin, cos, sqrt, atan2

# Load the IATA/ICAO CSV file into a DataFrame
airport_df = pd.read_csv('airports_au.csv')  # Replace with your local path

# Haversine formula to calculate distance between two lat/lon points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Find the nearest airport based on latitude and longitude
def get_nearest_airport_icao(lat, long):
    min_distance = float('inf')
    nearest_icao = 'UNKNOWN'
    nearest_name = 'UNKNOWN'
    
    for _, airport in airport_df.iterrows():
        # Ensure latitude and longitude are treated as floats
        try:
            airport_lat = float(airport['latitude_deg'])
            airport_long = float(airport['longitude_deg'])
            distance = haversine(float(lat), float(long), airport_lat, airport_long)
        except ValueError:
            continue  # Skip any rows where conversion to float fails
        
        if distance < min_distance:
            min_distance = distance
            nearest_icao = airport['ident']
            nearest_name = airport['name']
    
    return nearest_icao, nearest_name

# Main function to extract flight details and add start/end ICAO codes
def extract_flight_details(mgl_records, mgl_flight, mgl_filepath):
    # Add new columns to mgl_flight for start and end data
    mgl_flight['start_time'] = None
    mgl_flight['start_lat'] = None
    mgl_flight['start_long'] = None
    mgl_flight['end_time'] = None
    mgl_flight['end_lat'] = None
    mgl_flight['end_long'] = None
    mgl_flight['flight_time'] = None
    mgl_flight['start_icao'] = None
    mgl_flight['end_icao'] = None
    mgl_flight['start_name'] = None
    mgl_flight['end_name'] = None
    mgl_flight['file_path'] = None
    
    for i, flight_row in mgl_flight.iterrows():
        flight_guid = flight_row['Flight_GUID']
        matching_records = mgl_records[mgl_records['GUID'] == flight_guid]

        if not matching_records.empty:
            start_record = matching_records.iloc[0]
            end_record = matching_records.iloc[-1]

            start_time = start_record['Time']
            start_lat = float(start_record['lat'])  # Convert to float
            start_long = float(start_record['long'])  # Convert to float
            end_time = end_record['Time']
            end_lat = float(end_record['lat'])  # Convert to float
            end_long = float(end_record['long'])  # Convert to float
            flight_time = end_time - start_time

            # Find nearest ICAO codes for start and end
            start_icao, start_name = get_nearest_airport_icao(start_lat, start_long)
            end_icao, end_name = get_nearest_airport_icao(end_lat, end_long)

            # Update the DataFrame
            mgl_flight.at[i, 'start_time'] = start_time
            mgl_flight.at[i, 'start_lat'] = start_lat
            mgl_flight.at[i, 'start_long'] = start_long
            mgl_flight.at[i, 'end_time'] = end_time
            mgl_flight.at[i, 'end_lat'] = end_lat
            mgl_flight.at[i, 'end_long'] = end_long
            mgl_flight.at[i, 'flight_time'] = flight_time
            mgl_flight.at[i, 'start_icao'] = start_icao
            mgl_flight.at[i, 'end_icao'] = end_icao
            mgl_flight.at[i, 'start_name'] = start_name
            mgl_flight.at[i, 'end_name'] = end_name
            mgl_flight.at[i, 'file_path'] = mgl_filepath
        else:
            print(f"No matching records found for Flight_GUID: {flight_guid}")
    
    return mgl_flight
            

#Execution Starts Here

delimiter_type = 'comma'
filename =  'mgl/Enigma9Dec24' #'OAR-Enigma'
fileext = 'rec'
filepath = f"{filename}.{fileext}"

parser = EnigmaLogParser(filepath)
print("Scanning for first and last record, please wait")

#oldest_pos, newest_pos, sections_seen, reversals = parser.find_first_last()
sections_seen, start_time = parser.determine_sections()

print("Optional record sections present:")
if sections_seen & RDAC1_PRESENT:
    print(" RDAC1")
if sections_seen & RDAC2_PRESENT:
    print(" RDAC2")
if sections_seen & ATTITUDE_PRESENT:
    print(" Attitude")
if sections_seen & GPS_PRESENT:
    print(" GPS")

# Write the output to a file called "filename.txt"
with open(f"{filename}.txt", "w") as file:
    mgl_header = parser.print_header(delimiter_type, sections_seen)
    parser.bad_packets = 0
    # Get the current position
    parser.logfile.seek(0, 2)  # Seek to the end of the file
    file_size = parser.logfile.tell()    # Get the last position (i.e., the file size)
    print(f"File size: {file_size} bytes")
    parser.logfile.seek(0, 0)
    print(f"Dumping packets from:{parser.logfile.tell()} please wait")
    mgl_records, mgl_flight = parser.process_packets(delimiter_type, file_size, sections_seen, start_time)
# Creating a Pandas DataFrame using the headers and mgl_records
mgl_flight_df = pd.DataFrame(mgl_flight, columns=["Flight_GUID", "Start_Packet"])
mgl_record_df = pd.DataFrame(mgl_records, columns=mgl_header)
# Remove extra quotes from the 'Time' column, if they exist
mgl_record_df['Time'] = mgl_record_df['Time'].str.strip('"')
# Assuming your 'Time' column contains strings like "2024/09/20 12:00:00"
mgl_record_df['Time'] = pd.to_datetime(mgl_record_df['Time'], format="%Y/%m/%d %H:%M:%S")

mgl_flight_df = extract_flight_details(mgl_record_df, mgl_flight_df, filepath)

# Display the DataFrame
print(mgl_flight_df)
print(mgl_record_df)
parser.logfile.close()


Scanning for first and last record, please wait
Optional record sections present:
 RDAC1
 Attitude
 GPS
Time,GUID,alt,baro,ASI,TAS,VSI,g_s,rotor,m_v,b_v,amps,AOA,OAT,RPM,rfl1,rfl2,ch1,ch2,flow,MAP,fl1,fl2,flc,o_t,oil_p,carb,f_p,h2o,tc1,tc2,tc3,tc4,tc5,tc6,tc7,tc8,tc9,tc10,tc11,tc12,rch1,rch2,rot,rop,ref,fail,bank,ptch,slip,mgl_hdg,yaw,G,turn,lat,long,gps_hdg,gs,g_alt,status,sats,hac,vac
File size: 10485760 bytes
Dumping packets from:0 please wait
Flight: 74ec288e-75e0-4095-9dad-ff425c9f7e2b @ 2023/12/19 08:26:35
pFlight: 67ae1f42-0abe-458d-9a89-d819ac2327a5 @ 2023/12/20 09:38:02
pppppFlight: bc81c1d5-0783-4757-aed7-2a70ce3955e3 @ 2024/01/04 07:58:07
Flight: bd890973-ddf2-4035-8c2a-6a66db551f4d @ 2024/01/04 08:08:26
Flight: 41fa1675-d0c7-4330-a195-ec97654f0bcc @ 2024/01/04 09:29:56
pFlight: 11733484-1201-453c-b7e4-3126611beb1e @ 2024/01/04 11:54:13
ppFlight: e6701d51-7f9c-45e9-aead-fc4227afe9e2 @ 2024/01/07 09:43:41
ppFlight: 6809bacd-a664-40dd-bc17-5896f16289e0 @ 2024/01/07 17:55:01
pp

In [4]:
print(payload, payload_len, revision, sections_seen)


NameError: name 'payload' is not defined

In [ ]:
# Set display options to show all columns and avoid truncation
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)        # Set the display width to avoid wrapping/truncation
print(mgl_flight_df.head())  # Display the first 5 rows of mgl_flight_df
print(mgl_record_df.head())  # Display the first 5 rows of mgl_record_df


In [8]:
import folium
# Create a Folium map centered on Australia (latitude: -25.2744, longitude: 133.7751)
map_center = [-35.30131, 149.18610]
m = folium.Map(location=map_center, zoom_start=5)

# Loop through the DataFrame and add points to the map
for i, row in mgl_record_df.iloc[::360].iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=f"{row['Time']}",  # Customize popup as needed
        icon=folium.Icon(icon='info-sign', color='blue')  # Set icon and color
    ).add_to(m)

# Display the map (if you're in a Jupyter environment, this will render the map)
m.save('australia_map.html')  # Optionally, save the map to an HTML file

# To display in Jupyter Lab, simply return the map object

#print(mgl_record_df)
m


In [ ]:

import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Database connection details
host = "122.201.101.170"
user = "myearthj_steven"
password = "gurxyz-biske1-zuzwiS"
database = "myearthj_R2525"

# Establishing connection
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Verifying connection
if conn.is_connected():
    print("Connected to the database")
else:
    print("Connection failed")
    
def insert_without_duplicates(df, table_name, unique_column):
    # Connect to database
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"
    engine = create_engine(connection_string)

    # Get existing records based on the unique column
    existing_values = pd.read_sql(f"SELECT {unique_column} FROM {table_name}", con=engine)
    
    # Filter out rows from the DataFrame that already exist in the database
    df_to_insert = df[~df[unique_column].isin(existing_values[unique_column])]
    
    # Insert only the new records in chunks, printing progress
    if not df_to_insert.empty:
        batch_size = 1000
        for i in range(0, len(df_to_insert), batch_size):
            # Insert batch of records
            df_to_insert.iloc[i:i+batch_size].to_sql(table_name, con=engine, if_exists='append', index=False)
            
            # Print progress for every 1000 records
            print('.', end='', flush=True)
        
        print(f"\n{len(df_to_insert)} new records inserted into {table_name}.")
    else:
        print(f"No new records to insert into {table_name}.")


    # Close the connection
    engine.dispose()

    
# Convert datetime columns to MySQL-compatible datetime64 format
mgl_flight_df['start_time'] = pd.to_datetime(mgl_flight_df['start_time'])
mgl_flight_df['end_time'] = pd.to_datetime(mgl_flight_df['end_time'])
mgl_record_df['Time'] = pd.to_datetime(mgl_record_df['Time'])

# Convert timedelta columns to 'HH:MM:SS' format for MySQL TIME type
mgl_flight_df['flight_time'] = mgl_flight_df['flight_time'].apply(
    lambda x: str(x).split(' ')[-1] if pd.notnull(x) else None
)
# Use start_time for uniqueness in MGL_Flight and Time for MGL_Record
insert_without_duplicates(mgl_flight_df, 'MGL_Flight', 'start_time')
insert_without_duplicates(mgl_record_df, 'MGL_Record', 'Time')


# Remember to close the connection after finishing
conn.close()


In [ ]:
#Display Flight Map

import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import folium
from ipywidgets import widgets, interact

# Database connection details
host = "122.201.101.170"
user = "myearthj_steven"
password = "gurxyz-biske1-zuzwiS"
database = "myearthj_R2525"


# Create a connection engine using SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Load all flights into a DataFrame
flights_df = pd.read_sql("SELECT * FROM MGL_Flight ORDER BY start_time", engine)

# Create a dictionary for dropdown options using descriptive labels
flight_options = {
    f"{row['start_time']} | {row['start_icao']} -> {row['end_icao']}": row['Flight_GUID']
    for _, row in flights_df.iterrows()
}

# Create a dropdown widget to select a flight by descriptive label
flight_dropdown = widgets.Dropdown(
    options=flight_options,
    description='Select Flight:',
)

# Function to load the track points and display them on a map
def display_flight_track(flight_label):
    # Get the actual GUID, handling cases where flight_label is the GUID itself
    flight_guid = flight_options.get(flight_label, flight_label)
    
    # Query to get all track records for the selected flight
    query = f"SELECT * FROM MGL_Record WHERE GUID = '{flight_guid}'"
    track_df = pd.read_sql(query, engine)
    
    if track_df.empty:
        print("No records found for this flight.")
        return
    
    # Initialize the map at the start point of the track
    start_point = [track_df['lat'].iloc[0], track_df['long'].iloc[0]]
    flight_map = folium.Map(location=start_point, zoom_start=10)
    track_color = 'blue'
    # Add track points to the map, displaying every 60th record
    for idx, row in track_df.iloc[::60].iterrows():
        folium.CircleMarker(
            [row['lat'], row['long']],
            popup=f"Time: {row['Time']} Alt: {row['alt']}",
            radius=2,  # Adjust the size of the circle
            color=track_color,  # Outline color of the circle
            fill=True,
            fill_color=track_color,  # Fill color of the circle
            fill_opacity=0.5,  # Adjust the opacity of the fill
        ).add_to(flight_map)

    # Draw a line connecting all track points, using every 60th point
    folium.PolyLine(
        [(row['lat'], row['long']) for idx, row in track_df.iloc[::60].iterrows()],
        color="blue",
        weight=2.5,
    ).add_to(flight_map)


    # Display the map
    display(flight_map)

# Use interact to select a flight and display its track
interact(display_flight_track, flight_label=flight_dropdown)


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection details
host = "122.201.101.170"
user = "myearthj_steven"
password = "gurxyz-biske1-zuzwiS"
database = "myearthj_R2525"

# Create a connection engine using SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Query to select all MGL_Flight records with the count of linked MGL_Record entries
query = """
SELECT 
    f.start_icao AS Start_ICAO,
    f.end_icao AS End_ICAO,
    f.start_time AS Start_Time,
    COUNT(r.GUID) AS Record_Count,
    MAX(r.TAS) AS Max_TAS
FROM 
    MGL_Flight f
LEFT JOIN 
    MGL_Record r ON f.Flight_GUID = r.GUID
GROUP BY 
    f.Flight_GUID, f.start_icao, f.end_icao
ORDER BY 
    f.start_time;
"""

# Execute the query and load the result into a DataFrame
flight_record_counts = pd.read_sql(query, engine)

# Display the DataFrame
print(flight_record_counts)


In [ ]:
from sqlalchemy import create_engine, text



# Create a connection engine using SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Open a connection and execute deletion queries with commit
with engine.begin() as connection:
    connection.execute(text("DELETE FROM MGL_Record"))  # Delete from MGL_Record first
    connection.execute(text("DELETE FROM MGL_Flight"))  # Then delete from MGL_Flight

print("All records deleted from MGL_Record and MGL_Flight.")
